In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold

from keras.models import Model, Sequential
from keras.layers import Input, Dense, Concatenate, Reshape, Dropout
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [2]:
%matplotlib inline

In [44]:
data = pd.read_csv('data/avocado.csv')

In [43]:
data.T

,0,1,2,3,4,5,6,7,8,9,...,18239,18240,18241,18242,18243,18244,18245,18246,18247,18248
Unnamed: 0,0,1,2,3,4,5,6,7,8,9,...,2,3,4,5,6,7,8,9,10,11
Date,27-12-2015,20-12-2015,13-12-2015,06-12-2015,29-11-2015,22-11-2015,15-11-2015,08-11-2015,01-11-2015,25-10-2015,...,11-03-2018,04-03-2018,25-02-2018,18-02-2018,11-02-2018,04-02-2018,28-01-2018,21-01-2018,14-01-2018,07-01-2018
AveragePrice,1.33,1.35,0.93,1.08,1.28,1.26,0.99,0.98,1.02,1.07,...,1.56,1.54,1.57,1.56,1.57,1.63,1.71,1.87,1.93,1.62
Total Volume,64236.6,54877,118220,78992.1,51039.6,55979.8,83453.8,109428,99811.4,74338.8,...,22128.4,17393.3,18421.2,17597.1,15986.2,17074.8,13888,13766.8,16205.2,17489.6
4046,1036.74,674.28,794.7,1132,941.48,1184.27,1368.92,703.75,1022.15,842.4,...,2162.67,1832.24,1974.26,1892.05,1924.28,2046.96,1191.7,1191.92,1527.63,2894.77
4225,54454.8,44638.8,109150,71976.4,43838.4,48068,73672.7,101815,87315.6,64757.4,...,3194.25,1905.57,2482.65,1928.36,1368.32,1529.2,3431.5,2452.79,2981.04,2356.13
4770,48.16,58.33,130.5,72.58,75.78,43.61,93.26,80,85.34,113,...,8.93,0,0,0,0,0,0,727.94,727.01,224.53
Total Bags,8696.87,9505.56,8145.35,5811.16,6183.95,6683.91,8318.86,6829.22,11388.4,8625.92,...,16762.6,13655.5,13964.3,13776.7,12693.6,13498.7,9264.84,9394.11,10969.5,12014.1
Small Bags,8603.62,9408.07,8042.21,5677.4,5986.26,6556.47,8196.81,6266.85,11104.5,8061.47,...,16510.3,13401.9,13698.3,13553.5,12437.4,13066.8,8940.04,9351.8,10919.5,11988.1
Large Bags,93.25,97.49,103.14,133.76,197.69,127.44,122.05,562.37,283.83,564.45,...,252.25,253.56,266.06,223.18,256.22,431.85,324.8,42.31,50,26.01


In [42]:
data.isnull().sum()

Unnamed: 0      0
Date            0
AveragePrice    0
Total Volume    0
4046            0
4225            0
4770            0
Total Bags      0
Small Bags      0
Large Bags      0
XLarge Bags     0
type            0
year            0
region          0
dtype: int64

In [6]:
data['Day'], data['Month'] = data.Date.str[:2], data.Date.str[3:5]

In [7]:
data.T

,0,1,2,3,4,5,6,7,8,9,...,18239,18240,18241,18242,18243,18244,18245,18246,18247,18248
Unnamed: 0,0,1,2,3,4,5,6,7,8,9,...,2,3,4,5,6,7,8,9,10,11
Date,27-12-2015,20-12-2015,13-12-2015,06-12-2015,29-11-2015,22-11-2015,15-11-2015,08-11-2015,01-11-2015,25-10-2015,...,11-03-2018,04-03-2018,25-02-2018,18-02-2018,11-02-2018,04-02-2018,28-01-2018,21-01-2018,14-01-2018,07-01-2018
AveragePrice,1.33,1.35,0.93,1.08,1.28,1.26,0.99,0.98,1.02,1.07,...,1.56,1.54,1.57,1.56,1.57,1.63,1.71,1.87,1.93,1.62
Total Volume,64236.6,54877,118220,78992.1,51039.6,55979.8,83453.8,109428,99811.4,74338.8,...,22128.4,17393.3,18421.2,17597.1,15986.2,17074.8,13888,13766.8,16205.2,17489.6
4046,1036.74,674.28,794.7,1132,941.48,1184.27,1368.92,703.75,1022.15,842.4,...,2162.67,1832.24,1974.26,1892.05,1924.28,2046.96,1191.7,1191.92,1527.63,2894.77
4225,54454.8,44638.8,109150,71976.4,43838.4,48068,73672.7,101815,87315.6,64757.4,...,3194.25,1905.57,2482.65,1928.36,1368.32,1529.2,3431.5,2452.79,2981.04,2356.13
4770,48.16,58.33,130.5,72.58,75.78,43.61,93.26,80,85.34,113,...,8.93,0,0,0,0,0,0,727.94,727.01,224.53
Total Bags,8696.87,9505.56,8145.35,5811.16,6183.95,6683.91,8318.86,6829.22,11388.4,8625.92,...,16762.6,13655.5,13964.3,13776.7,12693.6,13498.7,9264.84,9394.11,10969.5,12014.1
Small Bags,8603.62,9408.07,8042.21,5677.4,5986.26,6556.47,8196.81,6266.85,11104.5,8061.47,...,16510.3,13401.9,13698.3,13553.5,12437.4,13066.8,8940.04,9351.8,10919.5,11988.1
Large Bags,93.25,97.49,103.14,133.76,197.69,127.44,122.05,562.37,283.83,564.45,...,252.25,253.56,266.06,223.18,256.22,431.85,324.8,42.31,50,26.01


In [8]:
data = data.drop(['Unnamed: 0', 'Date'], axis = 1)

In [9]:
label_dict = defaultdict(LabelEncoder)

In [11]:
data[['region', 'type', 'Day', 'Month', 'year']] = data[['region', 'type', 'Day', 'Month', 'year']].apply(lambda x: label_dict[x.name].fit_transform(x))

In [12]:
X = data
y = X.pop('AveragePrice')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=9)

In [13]:
X_train.shape

(12774, 13)

In [98]:
filepath="avocado-{epoch:02d}-{val_loss:.2f}.hdf5"
model_ckpt = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
es = EarlyStopping(monitor='val_loss', min_delta=1, patience=5, verbose=1)
callbacks = [model_ckpt, es]

In [99]:
model = Sequential()
model.add(Dense(units=16, activation='relu', input_dim=13))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1, activation='linear'))

model.compile(loss='mse', optimizer='adam')

In [100]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 16)                224       
_________________________________________________________________
dense_56 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_57 (Dense)             (None, 1)                 9         
Total params: 369
Trainable params: 369
Non-trainable params: 0
_________________________________________________________________


In [101]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=40, batch_size=32, callbacks = callbacks)

Train on 12774 samples, validate on 5475 samples
Epoch 1/40
12774/12774 [==============================] - 2s 126us/step - loss: 2807968453.0568 - val_loss: 10274988.9596

Epoch 00001: val_loss improved from inf to 10274988.95963, saving model to avocado-01-10274988.96.hdf5
Epoch 2/40
12774/12774 [==============================] - 1s 57us/step - loss: 9310287.0546 - val_loss: 4794927.5484

Epoch 00002: val_loss improved from 10274988.95963 to 4794927.54840, saving model to avocado-02-4794927.55.hdf5
Epoch 3/40
12774/12774 [==============================] - 1s 58us/step - loss: 4720091.1448 - val_loss: 2599052.6609

Epoch 00003: val_loss improved from 4794927.54840 to 2599052.66086, saving model to avocado-03-2599052.66.hdf5
Epoch 4/40
12774/12774 [==============================] - 1s 56us/step - loss: 2395575.2206 - val_loss: 1035682.3799

Epoch 00004: val_loss improved from 2599052.66086 to 1035682.37990, saving model to avocado-04-1035682.38.hdf5
Epoch 5/40
12774/12774 [=============


Epoch 00039: val_loss improved from 15.77628 to 14.02075, saving model to avocado-39-14.02.hdf5
Epoch 40/40
12774/12774 [==============================] - 1s 61us/step - loss: 2.1118 - val_loss: 11.9952

Epoch 00040: val_loss improved from 14.02075 to 11.99519, saving model to avocado-40-12.00.hdf5


In [102]:
model.evaluate(X_test, y_test)

5475/5475 [==============================] - 0s 26us/step


11.995191503812189